In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/Pakistan Project Data/HIES_2004_05_stata (1)/sec_c0.dta"
)
print(C.shape)
C.sample(5)


(91319, 21)


,hhcode,sec,msno,scq01,scq02,scq03,scqo4,scq05,scq06,scq07,...,scq08b,scq09a,scq09b,psu,quarter,region,psu_hies,hhno,hhcode_hies,province
43060,2.081001e+09,C,24,NaN,NaN,yes,0: less than class1,yes,class 1,3 private,...,NaN,NaN,NaN,20810007,1,urban,20210103,3,2.021010e+09,sindh
71741,3.172002e+09,C,1,no,no,no,NaN,NaN,NaN,NaN,...,NaN,10 employment,NaN,31720016,3,rural,31720301,1,3.172030e+09,nwfp
20410,1.182000e+09,C,9,NaN,NaN,yes,0: less than class1,yes,class 1,1 government,...,NaN,NaN,NaN,11820003,1,rural,11820101,4,1.182010e+09,punjab
22564,1.201002e+09,C,8,yes,yes,yes,class 5,no,NaN,NaN,...,NaN,13 parent did not approve,NaN,12010020,4,urban,10510403,10,1.051040e+09,punjab
38464,2.032001e+09,C,3,no,no,no,NaN,NaN,NaN,NaN,...,NaN,12 lack of male/female staff,13 parent did not approve,20320006,3,rural,20320301,1,2.032030e+09,sindh


In [2]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrrolled"] = C["scq05"]
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [15]:
C_relevant

,hhcode,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.001100e+09,yes,class 10,no,2 education completed,urban,punjab
1,1.001100e+09,yes,cl;ass 8,no,2 education completed,urban,punjab
2,1.001100e+09,yes,17: ma/msc,no,2 education completed,urban,punjab
3,1.001100e+09,yes,class 10,yes,NaN,urban,punjab
4,1.001100e+09,yes,12: ba/bsc,no,10 employment,urban,punjab
...,...,...,...,...,...,...,...
91314,4.262002e+09,no,NaN,NaN,15 other,rural,balochistan
91315,4.262002e+09,no,NaN,NaN,10 employment,rural,balochistan
91316,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan
91317,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan


In [4]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/Pakistan Project Data/HIES_2004_05_stata (1)/sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'age', 'sex', 'marital_status']].copy()

In [5]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [9]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode"], how='inner', validat, indicator=True)

# Evaluate merge 
years04_05._merge.value_counts()

both          771059
left_only          0
right_only         0
Name: _merge, dtype: int64

In [13]:
# Select people in school enrollment age (I would say 4-10 years old)
years04_05 = years04_05.loc[(years04_05["age"] >= 4)&(years04_05["age"] <= 15)]

In [16]:
years04_05.province.value_counts()

punjab         100091
nwfp            77612
sindh           63074
balochistan     44226
Name: province, dtype: int64